\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
Implement the forward propagation for a two-hidden-layer network for m-samples and n-features, as we discussed in class. Initialize the weights randomly. Use the data from the previous labs, such as logistic regression. You can choose the number of neurons in the hidden layer and use the sigmoid activation function. Report the evaluation metrics for the network.  Also, use other non-linear activation functions like ReLU and Tanh. Report the loss using both MSE and Cross Entropy.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("archive/weather_forecast_data.csv")
df

,Temperature,Humidity,Wind_Speed,Cloud_Cover,Pressure,Rain
0,23.720338,89.592641,7.335604,50.501694,1032.378759,rain
1,27.879734,46.489704,5.952484,4.990053,992.614190,no rain
2,25.069084,83.072843,1.371992,14.855784,1007.231620,no rain
3,23.622080,74.367758,7.050551,67.255282,982.632013,rain
4,20.591370,96.858822,4.643921,47.676444,980.825142,no rain
...,...,...,...,...,...,...
2495,21.791602,45.270902,11.807192,55.044682,1017.686181,no rain
2496,27.558479,46.481744,10.884915,39.715133,1008.590961,no rain
2497,28.108274,43.817178,2.897128,75.842952,999.119187,no rain
2498,14.789275,57.908105,2.374717,2.378743,1046.501875,no rain


In [2]:
df['Rain']=df['Rain'].map({'rain': 1, 'no rain': 0})
df.dropna(inplace=True)

X =df[['Temperature', 'Humidity', 'Wind_Speed', 'Cloud_Cover', 'Pressure']].values
y =df['Rain'].values.reshape(-1, 1)

#normalization
X_mean=np.mean(X, axis=0)
X_std=np.std(X, axis=0)
X=(X - X_mean) / X_std


In [3]:
# Activation functions and ot derivatives
def sigmoid(z):
    return 1/(1+np.exp(-z))
              
def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

def ReLU(z):
    return np.maximum(0, z)

def ReLU_derivative(z):
    return (z > 0).astype(float)

def Tanh(z):
    return np.tanh(z)

def tanh_derivative(z):
    return 1 - np.tanh(z)**2

# loss functions
def mse_loss(y_true, y_pred):
    m = y_true.shape[0]
    loss = np.sum((y_true - y_pred) ** 2) / m
    return loss
    
def cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-15  # to avoid log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    m = y_true.shape[0]
    loss = -np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)) / m
    return loss

# Initializing weights and bais
def initialize_weights(input_size, h1_size, h2_size, output_size):
    np.random.seed(42)  
    W1 = np.random.randn(input_size, h1_size) * 0.01
    b1 = np.zeros((1, h1_size))
    W2 = np.random.randn(h1_size, h2_size) * 0.01
    b2 = np.zeros((1, h2_size))
    W3 = np.random.randn(h2_size, output_size) * 0.01
    b3 = np.zeros((1, output_size))
    return W1, b1, W2, b2, W3, b3

In [4]:
# Forward propagation
def forward_propagation(X, W1, b1, W2, b2, W3, b3, activation):
    # Hidden layer 1
    Z1 = X @ W1 + b1
    A1 = activation(Z1)
    
    # Hidden layer 2
    Z2 = A1 @ W2 + b2
    A2 = activation(Z2)
    
    # Output layer
    Z3 = A2 @ W3 + b3
    A3 = sigmoid(Z3)  # final output using  sigmoid function
    return A3

In [5]:
def evaluate_model(X, y, activation_func):
    input_size = X.shape[1]
    h1, h2, out = 10, 5, 1
    
    # Initializing weights
    W1, b1, W2, b2, W3, b3 = initialize_weights(input_size, h1, h2, out)
    
    # Forward propagation
    y_pred = forward_propagation(X, W1, b1, W2, b2, W3, b3, activation_func)
    
    # Converting to binary labels
    y_pred_labels = (y_pred > 0.5).astype(int)
    
    # Losses
    mse = mse_loss(y, y_pred)
    ce = cross_entropy_loss(y, y_pred)
    
    # Accuracy
    def accuracy(y_true, y_pred):
        return np.mean(y_true == y_pred)
    acc = accuracy(y, y_pred_labels)
    
    # Evaluation metrics
    def evaluation_metrics(y_true, y_pred):
        TP = np.sum((y_true == 1) & (y_pred == 1))
        TN = np.sum((y_true == 0) & (y_pred == 0))
        FP = np.sum((y_true == 0) & (y_pred == 1))
        FN = np.sum((y_true == 1) & (y_pred == 0))

        accuracy = (TP + TN) / len(y_true)
        precision = TP / (TP + FP + 1e-10)
        recall = TP / (TP + FN + 1e-10)
        f1 = 2 * precision * recall / (precision + recall + 1e-10)

        print(f"Accuracy:  {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall:    {recall:.4f}")
        print(f"F1 Score:  {f1:.4f}")

        # return np.array([[TN, FP], [FN, TP]])  if we want confusion matrix 
    
    cm = evaluation_metrics(y, y_pred_labels)
    
    return mse, ce, acc, cm


In [7]:
activation_func = {
    "ReLU": ReLU,
    "Tanh": Tanh,
    "Sigmoid": sigmoid
}
for name, func in activation_func.items():
    mse, ce,acc,cm = evaluate_model(X, y, func)
    print(f"\nActivation: {name}")
    print(f"  - MSE Loss: {mse:.4f}")
    print(f"  - Cross-Entropy Loss: {ce:.4f}")
    print(f"  - Accuracy: {acc:.4f}")
    print(cm)
    print("-----------------------------")



Accuracy:  0.8744
Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000

Activation: ReLU
  - MSE Loss: 0.2500
  - Cross-Entropy Loss: 0.6931
  - Accuracy: 0.8744
None
-----------------------------
Accuracy:  0.6024
Precision: 0.2293
Recall:    0.9172
F1 Score:  0.3669

Activation: Tanh
  - MSE Loss: 0.2500
  - Cross-Entropy Loss: 0.6931
  - Accuracy: 0.6024
None
-----------------------------
Accuracy:  0.8744
Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000

Activation: Sigmoid
  - MSE Loss: 0.2471
  - Cross-Entropy Loss: 0.6873
  - Accuracy: 0.8744
None
-----------------------------
